In [8]:
import pandas as pd
import fastText as ft
import os
from sklearn.utils import shuffle
import numpy as np

In [2]:
df = pd.read_csv('amazon_ready_to_train.csv')

In [ ]:
df.head(10)

In [ ]:
#balance the dataset
l = []
for i in set(df.category):
    #for every unique microtag
    temp = df[df.category==i]
    #if count>10000, get only 10000 samples
    if len(temp)>10000:
        l.append(temp.sample(10000))
    else: #otherwise, get everything
        l.append(temp)


In [ ]:
#concatenate the dataset
df = pd.concat(l)
df = df[df.category.notnull()]

In [ ]:
df = shuffle(df, random_state = 100)

In [ ]:
df.shape

In [ ]:
df.to_csv("amazon_ready_to_train")

In [ ]:
df.head()

In [ ]:
#create fasttext specific training data
df['label']  = '__label__'+df.category.apply(lambda x: x.replace(' ','_')+' ')
df['label'] = df['label'] + df.product_name
#df[['label']].to_csv('training_data.txt', header=None, index=None, mode='a')



In [ ]:
#create fasttext unsupervised training data
df[['product_name']].to_csv('training_data_unsupervised.txt',header=None,index=None,mode='a')

In [ ]:
train_data = os.path.join(os.getenv("DATADIR", ''), 'training_data.txt')
train_data2 = os.path.join(os.getenv("DATADIR", ''), 'training_data_unsupervised.txt')

In [ ]:
#Train unsupervised model
model2 = ft.train_unsupervised

In [ ]:
#Train supervised model
model = ft.train_supervised(train_data, epoch=25, lr=1.0, wordNgrams=2, verbose=2, minCount=1)
model.save_model("amazon.bin")
model.

In [ ]:
#try several cases
print(model.predict('zara blue shirt'))
print(model.predict('calvin klein thermal underwear'))
print(model.predict('apple macbook pro'))

In [ ]:
model.quantize(input=train_data, qnorm=True, retrain=True, cutoff=100000)
model.save_model("amazon.ftz")

In [ ]:
#try several cases
print(model.predict('zara blue shirt'))
print(model.predict('calvin klein thermal underwear'))
print(model.predict('jack purcell low profile ox sneakers'))


In [ ]:
model.test('training_data.txt')

In [ ]:
model.get_sentence_vector('Rein')

In [4]:
model = ft.load_model('amazon.ftz')

In [15]:
df['sent_embed']  = df.product_name.apply(lambda x: model.get_sentence_vector(str(x)))

In [21]:
df.groupby('category').count()

,Unnamed: 0,brand,price,product_name,sent_embed
category,,,,,
active,10000,5886,10000,9992,10000
athletic shoes,10000,3367,9996,9999,10000
boots,10000,4211,9999,10000,10000
"coats, jackets and vests",10000,6451,9999,9998,10000
dresses,10000,8158,10000,9998,10000
fashion hoodies and sweatshirts,10000,8008,9999,9999,10000
fashion sneakers,10000,3052,9996,9999,10000
flats,10000,5130,10000,10000,10000
jeans,10000,5104,9996,10000,10000


In [22]:
df.head()

,Unnamed: 0,brand,category,price,product_name,sent_embed
0,125735,Domy,swimsuits and cover ups,22.99,plus size swimwear retro halter top vintage on...,"[-201.85323, -127.87654, 0.8215461, 323.24844,..."
1,93675,LE CHÂTEAU,pumps,49.97,women's classic leather pointy toe slingback pump,"[7.2089806, 124.064354, -22.363012, 173.3266, ..."
2,279434,DongDong,dresses,7.82,dress ladies short casual pocket summer sleeve...,"[39.89494, 102.13796, -127.38166, -96.415955, ..."
3,255335,NaN,jeans,105.00,super stretch skinny fit distressed jeans wfla...,"[40.05252, 124.04834, -93.65457, -474.3166, -1..."
4,92049,NaN,pumps,42.99,women's total motion salima dress pump,"[41.03999, 247.08846, 340.8324, -8.21388, -328..."


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,brand,category,price,product_name
0,0,0,0.0,TOMS,slippers,21.99,women's classics
1,1,1,1.0,Vionic,slippers,59.95,women's relax slipper (size 11/dark grey zebra)
2,2,2,2.0,Birkenstock,slippers,89.95,arizona women's birko-flor sandal
3,3,3,3.0,TOMS,slippers,19.99,men's classic canvas slip-on
4,4,4,4.0,ULTRAIDEAS,slippers,19.90,women's comfort memory foam slippers wool-like...
5,5,5,5.0,Shevalues,slippers,10.99,shower sandal slippers quick drying bathroom s...
6,6,6,6.0,NaN,slippers,13.99,women's comfort slip on memory foam slippers f...
7,7,7,7.0,Vionic,slippers,49.95,relax - orthaheel orthotic slippers
8,8,8,8.0,NaN,slippers,12.00,women's terry ballerina slipper with bow for i...
9,9,9,9.0,NaN,slippers,64.95,ohana slipper - women's blueberry/black 10
